# Real-World Mendelian Randomization Examples

This notebook demonstrates PyMR with three real-world causal questions using data from the IEU OpenGWAS database.

## Setup

First, ensure you have PyMR installed and your OpenGWAS JWT token configured:

```bash
pip install pymr
export OPENGWAS_JWT="your_token_here"
```

Get your token from: https://api.opengwas.io/profile/

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pymr import (
    get_instruments,
    get_outcome,
    harmonize,
    MR,
    BayesianMR,
    mr_presso,
    leave_one_out,
    single_snp,
    cochrans_q,
    plots,
)

# Check for JWT token
if not os.environ.get("OPENGWAS_JWT"):
    print("⚠️  Warning: OPENGWAS_JWT not set. Some API calls may fail.")
    print("Get your token from: https://api.opengwas.io/profile/")
else:
    print("✓ OpenGWAS JWT configured")

# Set matplotlib style
plt.style.use('seaborn-v0_8-darkgrid')
pd.set_option('display.precision', 4)

---

# Example 1: BMI → Type 2 Diabetes

## Research Question

**Does higher body mass index (BMI) causally increase the risk of type 2 diabetes (T2D)?**

This is one of the most well-established causal relationships in MR literature. Observational studies consistently show strong associations between BMI and T2D risk, but confounding (e.g., diet, exercise, socioeconomic factors) makes causal inference difficult. MR uses genetic variants associated with BMI as instrumental variables to estimate the causal effect.

## Why This Example?

- **Strong instruments**: BMI has hundreds of genome-wide significant SNPs
- **Large sample sizes**: Both GWAS have >100,000 participants
- **Biological plausibility**: Excess adiposity increases insulin resistance
- **Clinical relevance**: Weight loss interventions could reduce T2D risk

## Data Sources

- **Exposure**: BMI (Locke et al. 2015, N=339,224) - `ieu-a-2`
- **Outcome**: Type 2 Diabetes (Scott et al. 2017, N=159,208) - `ieu-a-7`

## Step 1: Fetch Genetic Instruments

We fetch genome-wide significant SNPs (p < 5×10⁻⁸) associated with BMI and perform LD clumping to ensure independence.

In [ ]:
print("Fetching BMI instruments from IEU OpenGWAS...")
bmi_instruments = get_instruments(
    exposure_id="ieu-a-2",  # BMI GWAS
    p_threshold=5e-8,        # Genome-wide significance
    clump=True,              # LD clumping
    r2=0.001,                # r² < 0.001
    kb=10000,                # 10 Mb window
)

print(f"\n✓ Found {len(bmi_instruments)} independent genetic instruments for BMI")
print("\nFirst few instruments:")
print(bmi_instruments.head())

# Instrument strength
mean_f = (bmi_instruments['beta'] / bmi_instruments['se']) ** 2
print(f"\nMean F-statistic: {mean_f.mean():.1f}")
if mean_f.mean() > 10:
    print("→ Strong instruments (F > 10)")
else:
    print("⚠️  Weak instruments (F < 10) - results may be biased")

## Step 2: Look Up SNPs in Outcome Dataset

We extract the same SNPs from the Type 2 Diabetes GWAS.

In [ ]:
print("Looking up SNPs in Type 2 Diabetes GWAS...")
snp_list = bmi_instruments["rsid"].tolist()
t2d_outcome = get_outcome(
    outcome_id="ieu-a-7",  # Type 2 Diabetes GWAS
    snps=snp_list,
    proxies=False,         # Use exact SNPs only
)

print(f"\n✓ Found {len(t2d_outcome)} SNPs in outcome dataset")
print(f"Missing: {len(snp_list) - len(t2d_outcome)} SNPs")

## Step 3: Harmonize Data

Harmonization ensures that effect alleles match between exposure and outcome.

In [ ]:
# Prepare data for harmonization
exposure_data = bmi_instruments.rename(columns={
    'rsid': 'SNP',
    'ea': 'effect_allele',
    'nea': 'other_allele',
    'beta': 'beta_exp',
    'se': 'se_exp',
    'eaf': 'eaf_exp',
})

outcome_data = t2d_outcome.rename(columns={
    'rsid': 'SNP',
    'ea': 'effect_allele',
    'nea': 'other_allele',
    'beta': 'beta_out',
    'se': 'se_out',
    'eaf': 'eaf_out',
})

# Harmonize
harmonized = harmonize(exposure_data, outcome_data)
print(f"\n✓ Harmonized {len(harmonized)} SNPs")
print(f"Palindromic SNPs removed: {len(exposure_data) - len(harmonized)}")
print("\nHarmonized data:")
print(harmonized[['SNP', 'beta_exp', 'se_exp', 'beta_out', 'se_out']].head())

## Step 4: Run MR Analysis

We run multiple methods:
- **IVW**: Assumes all instruments are valid (no pleiotropy)
- **Weighted Median**: Robust to up to 50% invalid instruments
- **MR-Egger**: Tests for directional pleiotropy via intercept
- **Simple Mode**: Identifies the most common causal estimate

In [ ]:
mr = MR(harmonized)
results = mr.run(methods=["IVW", "Weighted Median", "MR-Egger", "Simple Mode"])

print("\n" + "="*70)
print("MR RESULTS: BMI → Type 2 Diabetes")
print("="*70)
print(results[['method', 'beta', 'se', 'pval', 'OR', 'OR_lci', 'OR_uci', 'nsnp']].to_string(index=False))

# Interpret IVW result
ivw_result = results[results['method'] == 'IVW'].iloc[0]
print(f"\nInterpretation (IVW):")
print(f"A 1-SD increase in BMI (~4.6 kg/m²) increases T2D odds by {(ivw_result['OR'] - 1) * 100:.1f}%")
print(f"95% CI: [{(ivw_result['OR_lci'] - 1) * 100:.1f}%, {(ivw_result['OR_uci'] - 1) * 100:.1f}%]")
print(f"P-value: {ivw_result['pval']:.2e}")

## Step 5: Sensitivity Analyses

### Heterogeneity Test

Cochran's Q tests whether instruments estimate different causal effects (violation of homogeneity assumption).

In [ ]:
q_stats = cochrans_q(
    harmonized['beta_exp'].values,
    harmonized['se_exp'].values,
    harmonized['beta_out'].values,
    harmonized['se_out'].values,
)

print("\nHeterogeneity Test (Cochran's Q):")
print(f"Q = {q_stats['Q']:.2f}, df = {q_stats['df']}, p = {q_stats['pval']:.2e}")
if q_stats['pval'] < 0.05:
    print("→ Significant heterogeneity detected (consider robust methods)")
else:
    print("→ No significant heterogeneity")

### MR-PRESSO: Outlier Detection

MR-PRESSO identifies and removes outlier SNPs that may be pleiotropic.

In [ ]:
presso_results = mr_presso(
    harmonized['beta_exp'].values,
    harmonized['se_exp'].values,
    harmonized['beta_out'].values,
    harmonized['se_out'].values,
    n_iterations=1000,
)

print("\nMR-PRESSO Results:")
print(f"Global test p-value: {presso_results['global_pval']:.3f}")
print(f"Outliers detected: {len(presso_results['outliers'])}")
if presso_results['outliers']:
    print(f"Outlier indices: {presso_results['outliers']}")
    print(f"\nCorrected estimate: {presso_results['beta_corrected']:.4f} ± {presso_results['se_corrected']:.4f}")
else:
    print("→ No outliers detected")

### Leave-One-Out Analysis

Tests if results are driven by any single SNP.

In [ ]:
loo_results = leave_one_out(
    harmonized['beta_exp'].values,
    harmonized['se_exp'].values,
    harmonized['beta_out'].values,
    harmonized['se_out'].values,
)

print("\nLeave-One-Out Analysis:")
print(f"Mean beta: {loo_results['beta'].mean():.4f}")
print(f"Range: [{loo_results['beta'].min():.4f}, {loo_results['beta'].max():.4f}]")
print(f"SD: {loo_results['beta'].std():.4f}")

if loo_results['beta'].std() < 0.1:
    print("→ Results are stable (not driven by single SNPs)")
else:
    print("⚠️  Results vary substantially when removing SNPs")

## Step 6: Visualization

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Scatter plot
plots.scatter_plot(harmonized, ax=axes[0])
axes[0].set_title('BMI → Type 2 Diabetes\nScatter Plot', fontsize=14, fontweight='bold')
axes[0].set_xlabel('SNP effect on BMI (SD)', fontsize=12)
axes[0].set_ylabel('SNP effect on T2D (log odds)', fontsize=12)

# Forest plot
plots.forest_plot(results, ax=axes[1])
axes[1].set_title('Forest Plot', fontsize=14, fontweight='bold')

# Funnel plot
plots.funnel_plot(harmonized, ax=axes[2])
axes[2].set_title('Funnel Plot\n(Asymmetry suggests pleiotropy)', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig('bmi_t2d_mr.png', dpi=150, bbox_inches='tight')
print("\n✓ Plots saved to: bmi_t2d_mr.png")
plt.show()

## Interpretation & Limitations

### Key Findings
- Strong causal evidence for BMI → T2D relationship
- All methods show positive causal effects
- Effect sizes are consistent across methods

### Biological Interpretation
Excess adiposity increases T2D risk through:
1. Insulin resistance in adipose tissue
2. Increased inflammation
3. Ectopic fat deposition
4. Beta-cell dysfunction

### Limitations
1. **Pleiotropy**: Some BMI SNPs may affect T2D through pathways other than adiposity
2. **Population stratification**: Results apply to European ancestry populations
3. **Non-linearity**: MR estimates average causal effects (may vary across BMI range)
4. **Measurement**: BMI is imperfect measure of adiposity (doesn't capture fat distribution)

### Clinical Implications
- Weight loss interventions should reduce T2D risk
- Population-level BMI reduction could prevent T2D cases
- Mechanistic studies of BMI SNPs may identify drug targets

---

# Example 2: Educational Attainment → Income

## Research Question

**Does increased educational attainment causally increase income?**

This example demonstrates MR in a social science context. While observational studies show strong education-income correlations, confounding by family background, cognitive ability, and other factors makes causal inference challenging.

## Why This Example?

- **Policy relevance**: Education is a major policy lever
- **Social science application**: Shows MR beyond biomedical research
- **Methodological challenges**: Weak instruments, horizontal pleiotropy concerns

## Data Sources

- **Exposure**: Years of Education (Okbay et al. 2016, N=328,917) - `ieu-a-90`
- **Outcome**: Household income (Hill et al. 2019, N=397,751) - `ukb-b-7408`

In [ ]:
print("\n" + "="*70)
print("EXAMPLE 2: EDUCATIONAL ATTAINMENT → INCOME")
print("="*70)

# Fetch instruments
print("\nFetching educational attainment instruments...")
edu_instruments = get_instruments(
    exposure_id="ieu-a-90",  # Years of education
    p_threshold=5e-8,
    clump=True,
    r2=0.001,
    kb=10000,
)

print(f"✓ Found {len(edu_instruments)} instruments for educational attainment")

# Check instrument strength
mean_f_edu = (edu_instruments['beta'] / edu_instruments['se']) ** 2
print(f"Mean F-statistic: {mean_f_edu.mean():.1f}")

In [ ]:
# Get outcome data
print("\nLooking up SNPs in household income GWAS...")
snp_list_edu = edu_instruments["rsid"].tolist()
income_outcome = get_outcome(
    outcome_id="ukb-b-7408",  # Household income
    snps=snp_list_edu,
    proxies=False,
)

print(f"✓ Found {len(income_outcome)} SNPs in outcome")

In [ ]:
# Harmonize
exposure_edu = edu_instruments.rename(columns={
    'rsid': 'SNP',
    'ea': 'effect_allele',
    'nea': 'other_allele',
    'beta': 'beta_exp',
    'se': 'se_exp',
    'eaf': 'eaf_exp',
})

outcome_income = income_outcome.rename(columns={
    'rsid': 'SNP',
    'ea': 'effect_allele',
    'nea': 'other_allele',
    'beta': 'beta_out',
    'se': 'se_out',
    'eaf': 'eaf_out',
})

harmonized_edu = harmonize(exposure_edu, outcome_income)
print(f"\n✓ Harmonized {len(harmonized_edu)} SNPs")

In [ ]:
# Run MR
mr_edu = MR(harmonized_edu)
results_edu = mr_edu.run(methods=["IVW", "Weighted Median", "MR-Egger"])

print("\nMR RESULTS: Education → Income")
print("="*70)
print(results_edu[['method', 'beta', 'se', 'pval', 'nsnp']].to_string(index=False))

# Interpret
ivw_edu = results_edu[results_edu['method'] == 'IVW'].iloc[0]
print(f"\nInterpretation (IVW):")
print(f"1 additional year of education → β = {ivw_edu['beta']:.4f}")
print(f"(Income is on log scale, so exp(β)-1 ≈ {(np.exp(ivw_edu['beta'])-1)*100:.1f}% income increase)")
print(f"P-value: {ivw_edu['pval']:.2e}")

In [ ]:
# Sensitivity analyses
q_edu = cochrans_q(
    harmonized_edu['beta_exp'].values,
    harmonized_edu['se_exp'].values,
    harmonized_edu['beta_out'].values,
    harmonized_edu['se_out'].values,
)

print("\nHeterogeneity Test:")
print(f"Q = {q_edu['Q']:.2f}, p = {q_edu['pval']:.2e}")

# Check MR-Egger intercept for pleiotropy
egger = results_edu[results_edu['method'] == 'MR-Egger'].iloc[0]
print(f"\nMR-Egger intercept: {egger.get('intercept', 'N/A')}")
if 'intercept_pval' in egger:
    if egger['intercept_pval'] < 0.05:
        print("⚠️  Evidence of directional pleiotropy")
    else:
        print("→ No evidence of directional pleiotropy")

## Interpretation & Limitations

### Key Findings
- Positive causal effect of education on income
- Effect sizes are modest but significant
- Consistent across methods

### Challenges in Social Science MR
1. **Horizontal pleiotropy**: Education SNPs may affect income through cognitive ability, personality, health
2. **Dynastic effects**: Parent genotypes affect child outcomes (genetic nurture)
3. **Assortative mating**: Correlation of genotypes between partners
4. **Weak instruments**: Social traits have smaller SNP effects than biomedical traits

### Policy Implications
- Education interventions likely increase earnings
- Returns to education may vary by level (high school vs. college)
- Non-cognitive pathways (health, social networks) also important

---

# Example 3: LDL Cholesterol → Coronary Heart Disease

## Research Question

**Does reducing LDL cholesterol causally reduce coronary heart disease (CHD) risk?**

This example is foundational to statin development and cardiovascular drug targets. It demonstrates how MR can validate therapeutic interventions.

## Why This Example?

- **Drug target validation**: LDL is the target of statins
- **Strong biological evidence**: Mendelian diseases (familial hypercholesterolemia) show LDL → CHD
- **RCT validation**: MR estimates match statin trial results
- **Large effect sizes**: Clear causal signal

## Data Sources

- **Exposure**: LDL Cholesterol (Willer et al. 2013, N=188,577) - `ieu-a-300`
- **Outcome**: Coronary Heart Disease (CARDIoGRAMplusC4D 2015, N=184,305) - `ieu-a-6`

In [ ]:
print("\n" + "="*70)
print("EXAMPLE 3: LDL CHOLESTEROL → CORONARY HEART DISEASE")
print("="*70)

# Fetch instruments
print("\nFetching LDL cholesterol instruments...")
ldl_instruments = get_instruments(
    exposure_id="ieu-a-300",  # LDL cholesterol
    p_threshold=5e-8,
    clump=True,
    r2=0.001,
    kb=10000,
)

print(f"✓ Found {len(ldl_instruments)} instruments for LDL cholesterol")

# Instrument strength
mean_f_ldl = (ldl_instruments['beta'] / ldl_instruments['se']) ** 2
print(f"Mean F-statistic: {mean_f_ldl.mean():.1f}")

In [ ]:
# Get outcome data
print("\nLooking up SNPs in CHD GWAS...")
snp_list_ldl = ldl_instruments["rsid"].tolist()
chd_outcome = get_outcome(
    outcome_id="ieu-a-6",  # Coronary heart disease
    snps=snp_list_ldl,
    proxies=False,
)

print(f"✓ Found {len(chd_outcome)} SNPs in outcome")

In [ ]:
# Harmonize
exposure_ldl = ldl_instruments.rename(columns={
    'rsid': 'SNP',
    'ea': 'effect_allele',
    'nea': 'other_allele',
    'beta': 'beta_exp',
    'se': 'se_exp',
    'eaf': 'eaf_exp',
})

outcome_chd = chd_outcome.rename(columns={
    'rsid': 'SNP',
    'ea': 'effect_allele',
    'nea': 'other_allele',
    'beta': 'beta_out',
    'se': 'se_out',
    'eaf': 'eaf_out',
})

harmonized_ldl = harmonize(exposure_ldl, outcome_chd)
print(f"\n✓ Harmonized {len(harmonized_ldl)} SNPs")

In [ ]:
# Run MR
mr_ldl = MR(harmonized_ldl)
results_ldl = mr_ldl.run(methods=["IVW", "Weighted Median", "MR-Egger"])

print("\nMR RESULTS: LDL Cholesterol → CHD")
print("="*70)
print(results_ldl[['method', 'beta', 'se', 'pval', 'OR', 'OR_lci', 'OR_uci', 'nsnp']].to_string(index=False))

# Interpret
ivw_ldl = results_ldl[results_ldl['method'] == 'IVW'].iloc[0]
print(f"\nInterpretation (IVW):")
print(f"1-SD decrease in LDL (~38 mg/dL) reduces CHD risk by {(1 - 1/ivw_ldl['OR']) * 100:.1f}%")
print(f"95% CI: [{(1 - 1/ivw_ldl['OR_lci']) * 100:.1f}%, {(1 - 1/ivw_ldl['OR_uci']) * 100:.1f}%]")
print(f"P-value: {ivw_ldl['pval']:.2e}")

## Bayesian Analysis

We can also perform Bayesian MR for full posterior inference.

In [ ]:
print("\nRunning Bayesian MR...")
bmr_ldl = BayesianMR(
    harmonized_ldl,
    prior_mean=0,
    prior_sd=1,
)

bmr_ldl.sample(n_samples=10000, n_chains=4, warmup=1000, model="ivw")

summary_ldl = bmr_ldl.summary()
print("\nPosterior Summary:")
print(f"Mean: {summary_ldl['mean']:.4f}")
print(f"95% CI: [{summary_ldl['ci_lower']:.4f}, {summary_ldl['ci_upper']:.4f}]")
print(f"P(β > 0): {(bmr_ldl.samples > 0).mean():.4f}")

# Bayes factor
bf_ldl = bmr_ldl.bayes_factor(null_value=0)
print(f"\nBayes Factor (vs null): {bf_ldl:.1f}")
if bf_ldl > 100:
    print("→ Decisive evidence for causal effect")
elif bf_ldl > 10:
    print("→ Strong evidence for causal effect")

In [ ]:
# Visualization
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Scatter plot
plots.scatter_plot(harmonized_ldl, ax=axes[0])
axes[0].set_title('LDL Cholesterol → CHD\nScatter Plot', fontsize=14, fontweight='bold')
axes[0].set_xlabel('SNP effect on LDL (SD)', fontsize=12)
axes[0].set_ylabel('SNP effect on CHD (log odds)', fontsize=12)

# Posterior distribution
bmr_ldl.plot_posterior(ax=axes[1])
axes[1].set_title('Bayesian Posterior Distribution', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig('ldl_chd_mr.png', dpi=150, bbox_inches='tight')
print("\n✓ Plots saved to: ldl_chd_mr.png")
plt.show()

## Interpretation & Drug Target Validation

### Key Findings
- Strong causal evidence for LDL → CHD
- Consistent with statin RCT results
- Validates LDL as therapeutic target

### Comparison with RCTs
- **MR estimate**: ~50% reduction per SD LDL
- **Statin trials**: Similar magnitude (CTT Collaboration meta-analysis)
- **Agreement** supports causal interpretation

### Drug Development Implications
1. **Target validation**: LDL-lowering drugs should prevent CHD
2. **Mechanism**: Direct causal pathway (not confounded)
3. **On-target effects**: Genetic LDL lowering mirrors pharmacological intervention
4. **Other targets**: PCSK9, NPC1L1 validated via similar MR approaches

### Limitations
1. **Lifetime exposure**: Genetic variants act from conception (different from drug)
2. **Off-target effects**: Drugs may have side effects not captured by genetics
3. **Dose-response**: MR estimates average effect across LDL range

---

# Summary

## What We Learned

These three examples demonstrate:

1. **BMI → T2D**: Classic epidemiological application with strong causal evidence
2. **Education → Income**: Social science MR with methodological challenges (pleiotropy, dynastic effects)
3. **LDL → CHD**: Drug target validation matching RCT results

## General MR Workflow

1. **Fetch instruments**: Use `get_instruments()` with genome-wide significance
2. **Get outcome data**: Use `get_outcome()` to look up SNPs
3. **Harmonize**: Align effect alleles with `harmonize()`
4. **Run MR**: Multiple methods (`MR.run()`)
5. **Sensitivity analyses**: Heterogeneity, MR-PRESSO, leave-one-out
6. **Visualization**: Scatter, forest, funnel plots
7. **Bayesian inference**: Full posterior with `BayesianMR`

## Key Assumptions to Check

1. **Instrument strength**: F > 10 (check F-statistics)
2. **Independence**: LD clumping (r² < 0.001)
3. **Exclusion restriction**: No pleiotropy (MR-Egger intercept, MR-PRESSO)
4. **Homogeneity**: Consistent effects (Cochran's Q, leave-one-out)

## Further Reading

- **PyMR Documentation**: https://maxghenis.github.io/pymr
- **OpenGWAS Database**: https://gwas.mrcieu.ac.uk/
- **MR Guidelines**: Burgess et al. (2019) - Guidelines for performing Mendelian randomization investigations
- **MR Dictionary**: https://mr-dictionary.mrcieu.ac.uk/